In [1]:
from datetime import datetime, timedelta
from pandas import DataFrame, read_csv

### Loads and prepares the original dataset

In [2]:
# Loads the original data from the games played in May 2022
filename = '../data/real_positions_may2022.csv'  # same file as Data_Jeu_Pred-Prey_IAOMay2022/Data/All_Tracks.csv
cols = ['Elevation', 'Latitude', 'Longitude', 'Timestamp', 'Game', 'Player_ID']
df = read_csv(filename, usecols=cols)

# Converts the entries in column Timestamp into datetime objects
df['Timestamp'] = df.apply(lambda x: datetime.strptime(x['Timestamp'], '%Y-%m-%d  %H:%M:%S'), axis=1)

FileNotFoundError: [Errno 2] No such file or directory: '../data/real_positions_may2022'

### Generates the dataset with interpolated positions

In [ ]:
def interpolate_trajectory(df):
    """Returns a list of interpolated positions"""
    new_rows = []
    game = df['Game'].iloc[0]
    plid = df['Player_ID'].iloc[0]
    z0, x0, y0 = df['Elevation'].iloc[0], df['Latitude'].iloc[0], df['Longitude'].iloc[0]
    t0 = df['Timestamp'].iloc[0]
    for row in df[1:].itertuples():
        z1, x1, y1 = row.Elevation, row.Latitude, row.Longitude
        # print(z1, x1, y1)
        dt = row.delta
        for t in range(1, int(dt)):
            new_rows.append([z0 + (z1 - z0) * t / dt,
                             x0 + (x1 - x0) * t / dt,
                             y0 + (y1 - y0) * t / dt,
                             t0 + timedelta(seconds=t),
                             row.Game,
                             row.Player_ID,
                             'interpolated',
                             -1])
        z0, x0, y0 = z1, x1, y1
        t0 = row.Timestamp
    return new_rows

DataFrame.interpolate_trajectory = interpolate_trajectory

In [ ]:
# Identifies the original data points (to distinguish them from the interpotaled ones)
df['position'] = 'real'

# Computes the time between each real position
df['delta'] = df.groupby(['Game', 'Player_ID'])['Timestamp'].diff()
df['delta'] = df.apply(lambda x: x['delta'].total_seconds(), axis=1)
df['delta'].fillna(timedelta(seconds=0).total_seconds(), inplace=True)

# Extracts a list of all real positions, and appends the interpolated ones
data_list = df.values.tolist()
for name, sub_df in df.groupby(['Game', 'Player_ID']):
    data_list.extend(sub_df.interpolate_trajectory())

# Creates a new dataframe combining the real and the interpolated positions
new_df = DataFrame(data_list, columns=df.columns)
new_df.sort_values(by=['Game', 'Player_ID', 'Timestamp'], inplace=True)
new_df.drop(columns=['delta'], inplace=True)
new_df.reset_index(inplace=True, drop=True)

# Saves the interpotaled dataset to file
new_df.to_csv('../data/interpolated_positions_may2022.csv', index=False)

In [ ]:
new_df.head(20)